# Stats
public score: 0.6812705366922234

오차행렬:

 [[  758   212]
 
 [  253 10637]]

정확도: 0.9608

정밀도: 0.7498

재현율: 0.7814

F1: 0.7653

cv f1 score: 0.783

# 영업 성공 여부 분류 경진대회

## 1. 데이터 확인

### 필수 라이브러리

In [19]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    make_scorer,
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [21]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
from xgboost import XGBClassifier

### 데이터 셋 읽어오기

In [22]:
df_train = pd.read_csv("train.csv") # 학습용 데이터
df_test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [23]:
df_train.head() # 학습용 데이터 살펴보기

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


## 2. 데이터 전처리

### 레이블 인코딩

In [24]:
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)

    return series

In [25]:
# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

for col in label_columns:
    df_train[col] = df_train[col].apply(
        lambda x: x if pd.isna(x) else x.replace(" ", "").replace('-', '').upper()
    )
    df_test[col] = df_test[col].apply(
        lambda x: x if pd.isna(x) else x.replace(" ", "").replace('-', '').upper()
    )

df_all = pd.concat([df_train[label_columns], df_test[label_columns]])

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])

다시 학습 데이터와 제출 데이터를 분리합니다.

In [26]:
for col in label_columns:  
    df_train[col] = df_all.iloc[: len(df_train)][col]
    df_test[col] = df_all.iloc[len(df_train) :][col]

### 2-2. 학습, 검증 데이터 분리

In [27]:
used_features = [
    'bant_submit', 
    # 'customer_country',  # response_corporate을 대체, 사용안할 예정
    'business_unit',
    'com_reg_ver_win_rate', 
    'customer_idx', 
    'customer_type', 
    'enterprise',
    'historical_existing_cnt', 
    'id_strategic_ver', 
    'it_strategic_ver',
    'idit_strategic_ver', 
    'customer_job', 
    'lead_desc_length',
    'inquiry_type', 
    'product_category', 
    'product_subcategory',
    'product_modelname', 
    'customer_country.1', 
    'customer_position',
    'response_corporate', 
    'expected_timeline', 
    'ver_cus', 
    'ver_pro',
    'ver_win_rate_x', 
    'ver_win_ratio_per_bu', 
    'business_area',
    'business_subarea', 
    'lead_owner', 
    # 'is_converted'
]

In [28]:
x_train, x_val, y_train, y_val = train_test_split(
    df_train.drop("is_converted", axis=1),
    df_train["is_converted"],
    test_size=0.2,
    shuffle=True,
    stratify=df_train['is_converted'],
    random_state=42,
)

## 3. 모델 학습

### 모델 정의 

In [29]:
train_df_vc = df_train['is_converted'].value_counts()
scale_pos_weight = train_df_vc.iloc[0] / train_df_vc.iloc[1]

model = XGBClassifier(
    n_estimators=101,
    max_depth=6,  # 일반적으로 3 ~ 10
    learning_rate=0.11183860093330873,
    subsample=1.0,  # 일반적으로 0.5 ~ 1
    colsample_bytree=0.9090606434532821,  # 일반적으로 0.5 ~ 1
    scale_pos_weight=2.93352578649596,  # sum(neg) / sum(pos)로 변경해보기
    reg_lambda=1.0966461771613576,  # L2-regularization
    max_leaves=0,
    max_bin=256,
    grow_policy=None,
    verbosity=0,
    objectjve='binary:logistic',
    booster='gbtree',
    tree_method='auto',  # XGBoost가 가장 conservative하게 결정
    n_jobs=-1,
    gamma=0,
    min_child_weight=1,
    max_delta_step=None,
    sampling_method=None,
    colsample_bylevel=1.0,
    colsample_bynode=1.0,
    reg_alpha=0,  # L1-regularization
    base_score=None,  # XGBoost에서 알아서 설정함
    random_state=42,
    missing=np.nan,  # 주어진 데이터셋에서 결측치 지정
    num_parallel_tree=1,  # boosted RF에서만 사용됨
    importance_type='weight',
    device='cpu',
    validate_parameters=True,  # unknown parameter에 대해 경고
    # enable_categorical=True,
    eval_metric=make_scorer(f1_score),  # early stopping에 사용됨
    early_stopping_rounds=None
)

### 모델 학습

In [30]:
model.fit(x_train.fillna(0), y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=1.0, colsample_bynode=1.0,
              colsample_bytree=0.9090606434532821, device='cpu',
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=make_scorer(f1_score), feature_types=None, gamma=0,
              grow_policy=None, importance_type='weight',
              interaction_constraints=None, learning_rate=0.11183860093330873,
              max_bin=256, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=0,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=101, n_jobs=-1,
              num_parallel_tree=1, objectjve='binary:logistic', ...)

### 모델 성능 보기

In [31]:
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))

In [32]:
pred = model.predict(x_val.fillna(0))
get_clf_eval(y_val, pred)

오차행렬:
 [[  758   212]
 [  253 10637]]

정확도: 0.9608
정밀도: 0.7498
재현율: 0.7814
F1: 0.7653


In [33]:
cv_scores = cross_val_score(
    model,
    df_train.drop('is_converted', axis=1).fillna(0),
    df_train['is_converted'],
    cv=StratifiedKFold(n_splits=3, shuffle=True, random_state=42),
    scoring=make_scorer(f1_score)
)
print(cv_scores)
print(cv_scores.mean().round(3))

[0.78549665 0.78461538 0.78797272]
0.786


## 4. 제출하기

### 테스트 데이터 예측

In [34]:
# 예측에 필요한 데이터 분리
x_test = df_test.drop(["is_converted", "id"], axis=1)

In [35]:
test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

1914

### 제출 파일 작성

In [36]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**